In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers accelerate bitsandbytes

Looking in indexes: https://download.pytorch.org/whl/cu118
   ---------------------------------------- 0.0/69.1 MB ? eta -:--:--
   ----- ---------------------------------- 9.2/69.1 MB 57.1 MB/s eta 0:00:02
   ----------------- ---------------------- 29.9/69.1 MB 82.3 MB/s eta 0:00:01
   ----------------------------- ---------- 50.3/69.1 MB 91.5 MB/s eta 0:00:01
   ---------------------------------------  68.9/69.1 MB 95.5 MB/s eta 0:00:01
   ---------------------------------------- 69.1/69.1 MB 81.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests
from PIL import Image
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from IPython.display import display_markdown
# from llama_index import SimpleDirectoryReader, GPTVectorStoreIndex

In [8]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Device name: NVIDIA GeForce RTX 4080 SUPER


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
device = torch.device("cuda", torch.cuda.current_device())
model = model.half().cuda()
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [10]:
def generate_chat_template(model, tokenizer, sys_pmt = "", prompt = "", temperature=0.7, top_p = 0.9, max_new_tokens=1000, do_sample = True):
    #sys_pmt = "Your task is to generate a brief summary of a given text, providing the main points and key details while maintaining coherence and clarity."
    messages = [
        {"role": "system", "content": sys_pmt},
        {"role": "user", "content": prompt},
    ]
    
    pipeline_ = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
    terminators = [pipeline_._tokenizer.eos_token_id, 
                   pipeline_._tokenizer.convert_tokens_to_ids("<eot_id>")]

    prompt = pipeline_._tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    outputs = pipeline_(
        prompt, 
        max_new_tokens=max_new_tokens,
        eos_token_id=terminators, 
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p
    )
    
    return outputs[0]["generated_text"].split(prompt)[1]



In [11]:
# Paraphrasing Prompt Basic Example
system_prompt = """
You are a basic paraphrasing tool.
"""

prompt = """
Paraphrase the following text:

Central Analytics Support & Enablement (CASE) is our enterprise support platform for the ~20k data scientists, data analysts, MLOps engineers, BI developers, and other support teams. CASE includes multiple features such as "CASEY" the chatbot, an intent-driven (i.e. non-Gen AI/non-LLM) chatbot microservice built on MS Copilot SDKs. Additionally, CASE includes the Collective Intelligence (CI) framework, conditional ticketing system, and MS Graph-enabled search. All of this enables product and support teams to efficiently address customer needs, manage their product knowledge documentation, and handle product-related issues.
"""

output = generate_chat_template(
    model=model,
    tokenizer=tokenizer,
    sys_pmt=system_prompt,
    prompt=prompt,
    temperature=0.7,  # Allows for more variation
    top_p=0.9, # Nucleus sampling
    do_sample=True    # Enables sampling
)
display_markdown(output, raw=True)


Device set to use cuda:0


AttributeError: 'TextGenerationPipeline' object has no attribute '_tokenizer'